In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import os.path
import pandas as pd
import numpy as np

#### Selecionando a lista de ações através de um dataset de ações da B3

In [27]:
df_acoes = pd.read_csv('arquivos/acoes-listadas-b3.csv')
todas_as_acoes = df_acoes['Ticker'].to_list()
todas_as_acoes_nparray = np.array(todas_as_acoes)
#selecione as 300 primeiras ações
lista_acoes2 = todas_as_acoes_nparray[0:300]


In [28]:
lista_acoes = np.array([]) # Aqui eu colei todos os nomes das ações e, a medida que erros iam acontecendo, eu ia removendo os nomes das ações que já tinham sido inseridas no csv

len(lista_acoes)

0

#### Código do bot que visitava o site investidor10 para coletar os resultados trimestrais das empresas

In [29]:
def bot():
    servico = Service(ChromeDriverManager().install())
    lista_acoes_validas = []
    acoes_com_erro = []
    navegador = webdriver.Chrome(service=servico)
    navegador.maximize_window()
    for acao in lista_acoes:


        navegador.get(f"https://investidor10.com.br/acoes/{acao}/")

        tabela_resultados = navegador.find_element(By.ID, "results_table")
        # desse a tela ate o elemento
        navegador.execute_script("arguments[0].scrollIntoView();", tabela_resultados)
        time.sleep(2)
        # verifica se ta visivil
        while not tabela_resultados.is_displayed():
            navegador.execute_script("arguments[0].scrollIntoView();", tabela_resultados)

        time.sleep(2)

        botoes = tabela_resultados.find_element(By.CLASS_NAME, "nav")
        botoes = botoes.find_elements(By.TAG_NAME, "li")


        bt_trimestral = botoes[3]
        bt_trimestral.click()
        time.sleep(1)

        dropdow = navegador.find_element(By.CLASS_NAME, "select2-results__options")
        dropdow = dropdow.find_elements(By.TAG_NAME, "li")
        dropdow[1].click()
        time.sleep(1)

        bt_valores = botoes[1]
        bt_valores.click()
        time.sleep(0.5)
        # clica no botao onde tem o texto "Valores Detalhados"
        dropdow = navegador.find_element(By.CLASS_NAME, "select2-results__options")
        dropdow = dropdow.find_elements(By.TAG_NAME, "li")
        dropdow[1].click()

        time.sleep(1)

        tabela = navegador.find_element(By.ID, "table-balance-results")

        linhas = tabela.find_elements(By.TAG_NAME, "tr")



        # A partir daqui, os dados estao sendo coletados
        # salve esses dados em um arquivo CSV
        colunas = linhas[0].find_elements(By.TAG_NAME, "th")
        if not os.path.isfile('arquivos/dataset_ML.csv'): # Para nao escrever as colunas novamente
            with open('arquivos/dataset_ML.csv', 'a', newline='', encoding='utf-8') as f:
                f.write("Empresa;Resultado;")
                for coluna in colunas:
                    if coluna.text == "" or coluna.text == "#":
                        continue
                    # inserindo uma condiçao para nao colocar o ";" na ultima coluna
                    if coluna.text == "1T2019":
                        linha = coluna.text + "\n"
                    else:
                        linha = coluna.text + ";"
                    f.write(linha)
                    print(coluna.text, end=" ")
                print()
                f.close()
        with open('arquivos/dataset_ML.csv', 'a', newline='', encoding='utf-8') as f:
            indices_desejados = [1,4,6,9,13]
            for idx, linha in enumerate(linhas[1:], start=1):
                if idx in indices_desejados:
                    colunas = linha.find_elements(By.TAG_NAME, "td")
                    i = 1
                    if len(colunas) !=60: #Significa que estão faltando trimestres nos dados da ação
                        acoes_com_erro.append(acao)
                        break #Passo para a próxima ação
                    f.write(acao + ";")
                    if acao not in lista_acoes_validas:
                        lista_acoes_validas.append(acao)
                    for coluna in colunas:
                        
                        if coluna.text == "":
                            continue
                        #faça uma condicao que verifique coluna.text é o ultimo elemento e nao coloque o ";"
                        
                        if i == 21:
                            linha = coluna.text + "\n"
                        else:
                            linha = coluna.text + ";"
                        i += 1
                        f.write(linha)

                        print(coluna.text, end=" ")
            
                    print()

    lista_acoes = lista_acoes_validas
    return lista_acoes, acoes_com_erro

### Funções usadas para criar o dataset final, que contém os resultados trimestrais de 2023 e uma classificação: 
- 0 = ação desvalorizou.
- 1 = ação valorizou menos de 10%.
- 2 = ação valorizou mais de 10%.

In [30]:
def criacao_dataframe_3trimestres(df, index_col, columns_col, reindex_list, ano):
    pivot_table = df.pivot_table(index=index_col, columns=columns_col, aggfunc='sum')
    pivot_table = pivot_table.reindex(reindex_list)
    pivot_table.columns = ['_'.join(col).strip() for col in pivot_table.columns.values] # Transformando o multi index da coluna em um index simples
    pivot_table = pivot_table.filter(like=ano, axis=1) # Filtrando apenas os resultados do ano passado na funçao
    pivot_table = pivot_table.drop(pivot_table.filter(like='4T').columns, axis=1) # Removendo os resultados do 4T de 2023

    # Removendo as linhas que possuem alguma celula com um '-'
    pivot_table = pivot_table.replace('-', np.nan)
    pivot_table = pivot_table.dropna()
    acoes_presentes_no_dataframe = np.array(pivot_table.index) #Obtendo as ações do dataframe após a limpeza das que possuíam dados faltantes para ser usada na função criacao_dataframe_dados_historicos
    # Criando novos nomes de colunas para o dataframe
    colunas = ['1T_Dívida Líquida', '1T_EBIT', '1T_Lucro Líquido', '1T_ROE', '1T_Receita Líquida', '2T_Dívida Líquida', '2T_EBIT', '2T_Lucro Líquido','2T_ROE', '2T_Receita Líquida', '3T_Dívida Líquida', '3T_EBIT', '3T_Lucro Líquido', '3T_ROE', '3T_Receita Líquida']
    pivot_table.columns = colunas
    # Adicionando a string ' - 2023' ao index do dataframe
    pivot_table.index = pivot_table.index + ' - ' + ano

    return pivot_table, acoes_presentes_no_dataframe

"""Essa função cria uma tabela pivot a partir de um dataframe, onde index_col é a coluna que será utilizada como índice (Empresa), columns_col é a coluna que será utilizada como coluna (Resultado) e reindex_list é a lista que será utilizada para 
reindexar a tabela pivot (lista_acoes),pois a ordem original foi alterada. Eu estou colocando como índice cada empresa e como coluna os resultados de cada trimestre (menos o 4T), filtrando pelo ano de 2023, que servirá como base de dados para análise"""

'Essa função cria uma tabela pivot a partir de um dataframe, onde index_col é a coluna que será utilizada como índice (Empresa), columns_col é a coluna que será utilizada como coluna (Resultado) e reindex_list é a lista que será utilizada para \nreindexar a tabela pivot (lista_acoes),pois a ordem original foi alterada. Eu estou colocando como índice cada empresa e como coluna os resultados de cada trimestre (menos o 4T), filtrando pelo ano de 2023, que servirá como base de dados para análise'

In [31]:
def criacao_dataframe_dados_historicos(diretorio, lista_acoes, mes_de_valorizacao, ano):
    dataframes = []
    for acao in lista_acoes:
        df = pd.read_csv(f'{diretorio}/{acao} Dados Históricos.csv')
        #Inserindo o nome da empresa na coluna "Empresa" e colocando ela como index
        df['Empresa'] = acao
        df.set_index('Empresa', inplace=True)
        # Selecionando apenas o mes de novembro de 2023
        df = df[df['Data'] == mes_de_valorizacao]
        # Calculando a variação - convertendo os valores em float
        df['Último'] = df['Último'].str.replace(',', '.').astype(float)
        df['Abertura'] = df['Abertura'].str.replace(',', '.').astype(float)
        df['Variação'] = (df['Último'] - df['Abertura']) / df['Abertura'] * 100
        # Criando a classificação, que vai servir como o Y do modelo
        if (df['Variação'] > 0).any():
            #if (df['Variação'] > 10).any():
                #df['Classificação'] = 2
            #else:
            df['Classificação'] = 1
        else:
            df['Classificação'] = 0
        # Adicionando a string ' - 2023' ao index do dataframe
        df.index = df.index + ' - ' + ano
        dataframes.append(df)
    df_final = pd.concat(dataframes)
    return df_final

"""Essa função está selecionando os arquivos csv dentro da pasta 'dados historicos' de acorco com a lista_acoes. Para cada arquivo csv, ela está inserindo o nome da empresa na coluna "Empresa" e colocando ela no começo do dataframe, 
selecionando apenas o mês de novembro de 2023, calculando a variação da ação nesse mês e criando a classificação, que vai servir como o Y do modelo. Se a empresa caiu, a classificação é 0, se subiu até 10%, a classificação é 1, 
e se subiu mais de 10%, a classificação é 2."""

'Essa função está selecionando os arquivos csv dentro da pasta \'dados historicos\' de acorco com a lista_acoes. Para cada arquivo csv, ela está inserindo o nome da empresa na coluna "Empresa" e colocando ela no começo do dataframe, \nselecionando apenas o mês de novembro de 2023, calculando a variação da ação nesse mês e criando a classificação, que vai servir como o Y do modelo. Se a empresa caiu, a classificação é 0, se subiu até 10%, a classificação é 1, \ne se subiu mais de 10%, a classificação é 2.'

In [32]:
def atribuir_classificacao(dataframe_resultados, dataframe_dados_historicos):
    dataframe_final = dataframe_resultados
    dataframe_final['Classificação'] = dataframe_dados_historicos['Classificação']
    return dataframe_final

# Essa função está atribuindo a coluna classificação do dataframe de dados históricos ao dataframe de resultados

Sobre as colunas do dataset:
- **Receita Líquida:** é a quantia de dinheiro que uma empresa ganha com suas vendas após a dedução de devoluções, descontos e impostos sobre vendas.
- **Lucro Líquido:** representa o lucro de uma empresa após todas as despesas e impostos terem sido deduzidos da receita total.
- **EBIT:** representa a lucratividade de uma empresa antes da consideração dos juros e impostos.
- **Dívida Líquida:** a diferença entre todos os compromissos financeiros pendentes e os recursos financeiros imediatamente disponíveis para liquidá-los.
- **ROE:** é uma medida que revela a capacidade de uma empresa gerar lucro a partir do dinheiro que os acionistas investiram nela.


In [33]:
# Ler o arquivo CSV
df = pd.read_csv('arquivos/dataset_ML.csv', delimiter=';')
df2 = df.copy()

## Criando o dataframe em que cada coluna representa um resultado de um trimestre

# 2023

In [34]:
#Obtendo os valores unicos da coluna 'Empresa' para atribuir para lista_acoes
lista_acoes = df2['Empresa'].unique()
dataframe_3trimestres_2023, acoes_do_dataframe_2023 = criacao_dataframe_3trimestres(df2, 'Empresa', 'Resultado', lista_acoes, '2023')
dataframe_3trimestres_2023


,1T_Dívida Líquida,1T_EBIT,1T_Lucro Líquido,1T_ROE,1T_Receita Líquida,2T_Dívida Líquida,2T_EBIT,2T_Lucro Líquido,2T_ROE,2T_Receita Líquida,3T_Dívida Líquida,3T_EBIT,3T_Lucro Líquido,3T_ROE,3T_Receita Líquida
Empresa,,,,,,,,,,,,,,,
PETR4 - 2023,204.125.000.000,60.197.000.000,38.307.000.000,"9,49 %",139.068.000.000,218.390.000.000,41.971.000.000,28.936.000.000,"7,76 %",113.840.000.000,238.304.000.000,47.546.000.000,26.760.000.000,"6,90 %",124.828.000.000
MGLU3 - 2023,5.038.335.000,16.328.000,-391.221.000,"-3,80 %",9.067.334.000,4.744.924.000,-35.923.000,-301.746.000,"-3,01 %",8.572.256.000,4.118.464.000,661.189.000,-498.330.000,"-5,23 %",8.578.818.000
OIBR3 - 2023,20.956.798.000,-102.677.000,-1.267.087.000,"-5,47 %",2.535.792.000,21.232.781.000,-266.082.000,-844.524.000,"-3,53 %",2.454.171.000,22.719.925.000,-803.758.000,-2.830.492.000,"-10,55 %",2.421.776.000
B3SA3 - 2023,-2.501.944.000,1.356.975.000,1.089.458.000,"5,30 %",2.460.480.000,-2.834.396.000,1.374.315.000,1.052.942.000,"5,13 %",2.477.122.000,-3.353.958.000,1.347.206.000,1.074.307.000,"5,31 %",2.490.082.000
COGN3 - 2023,6.158.625.000,259.730.000,53.920.000,"0,45 %",1.329.876.000,6.161.440.000,147.387.000,-64.907.000,"-0,39 %",1.386.677.000,6.077.436.000,90.916.000,-116.656.000,"-0,85 %",1.270.040.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAHI3 - 2023,177.193.000,-7.610.000,-17.872.000,"-30,31 %",86.779.000,167.306.000,1.138.000,-14.127.000,"-22,00 %",100.934.000,178.597.000,-597.000,-15.005.000,"-45,80 %",94.225.000
NEXP3 - 2023,48.337.000,-13.298.000,-17.843.000,"-18,42 %",30.066.000,50.187.000,26.464.000,29.871.000,"-23,20 %",37.420.000,50.369.000,4.650.000,4.411.000,"-4,09 %",-2.473.000
EMAE4 - 2023,-447.375.000,1.038.000,18.882.000,"1,65 %",137.552.000,-468.834.000,10.281.000,37.848.000,"3,39 %",138.754.000,-457.016.000,39.526.000,41.102.000,"3,55 %",173.596.000


# 2022

In [35]:
#Obtendo os valores unicos da coluna 'Empresa' para atribuir para lista_acoes
lista_acoes = df2['Empresa'].unique()
dataframe_3trimestres_2022, acoes_do_dataframe_2022 = criacao_dataframe_3trimestres(df2, 'Empresa', 'Resultado', lista_acoes, '2022')
dataframe_3trimestres_2022

,1T_Dívida Líquida,1T_EBIT,1T_Lucro Líquido,1T_ROE,1T_Receita Líquida,2T_Dívida Líquida,2T_EBIT,2T_Lucro Líquido,2T_ROE,2T_Receita Líquida,3T_Dívida Líquida,3T_EBIT,3T_Lucro Líquido,3T_ROE,3T_Receita Líquida
Empresa,,,,,,,,,,,,,,,
PETR4 - 2022,189.848.000.000,65.398.000.000,44.783.000.000,"10,26 %",141.641.000.000,180.367.000.000,96.434.000.000,54.484.000.000,"13,23 %",170.960.000.000,256.715.000.000,74.612.000.000,46.236.000.000,"12,37 %",170.076.000.000
OIBR3 - 2022,28.541.960.000,-369.370.000,1.783.950.000,"172,14 %",2.486.940.000,16.167.680.000,-611.400.000,-320.660.000,"-41,52 %",2.579.520.000,18.365.367.000,-992.420.000,-3.062.460.000,"-134,16 %",2.770.060.000
B3SA3 - 2022,-5.093.250.000,1.430.770.000,1.101.040.000,"5,05 %",2.284.700.000,-2.477.130.000,1.399.600.000,1.092.280.000,"5,29 %",2.241.630.000,-2.874.606.000,1.415.720.000,1.030.000.000,"5,04 %",2.510.000.000
ITSA4 - 2022,4.888.000.000,4.167.000.000,3.859.000.000,"5,67 %",2.131.000.000,5.137.000.000,3.270.000.000,3.181.000.000,"4,56 %",2.214.000.000,7.387.000.000,3.961.000.000,3.651.000.000,"5,02 %",2.161.000.000
ABEV3 - 2022,-11.247.310.000,4.162.330.000,3.528.840.000,"4,32 %",18.439.150.000,-12.468.933.000,4.034.080.000,3.064.050.000,"3,44 %",17.989.000.000,-16.079.870.000,4.054.970.000,3.215.010.000,"3,45 %",20.587.640.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BOBR4 - 2022,293.040.000,2.590.000,-24.700.000,"-12,17 %",310.110.000,330.890.000,38.460.000,7.950.000,"-4,08 %",371.260.000,308.475.000,51.200.000,15.740.000,"-8,78 %",387.150.000
CRPG5 - 2022,-134.400.000,-6.110.000,-1.030.000,"-0,12 %",210.140.000,-137.060.000,-770.000,4.730.000,"0,54 %",221.180.000,-155.452.000,19.830.000,28.440.000,"3,12 %",216.090.000
BAHI3 - 2022,59.430.000,-6.760.000,-18.070.000,"-9,61 %",83.750.000,122.337.000,-16.190.000,-35.550.000,"-25,86 %",86.870.000,198.932.000,-7.818.000,-28.105.000,"-23,99 %",90.803.000


## Criando o dataframe que contém a variação do preço das ações em novembro, e que vai gerar coluna classificação 

# 2023

In [36]:
dataframe_variacao_preco_acoes_2023 = criacao_dataframe_dados_historicos('dados historicos', acoes_do_dataframe_2023, '01.11.2023', '2023')
dataframe_variacao_preco_acoes_2023

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Variação,Classificação
Empresa,,,,,,,,,
PETR4 - 2023,01.11.2023,35.91,33.73,"36,03","32,66","992,20M","3,37%",6.463089,1
MGLU3 - 2023,01.11.2023,2.02,1.33,"2,35","1,31","4,53B","51,88%",51.879699,1
OIBR3 - 2023,01.11.2023,0.65,0.60,"0,75","0,59","120,11M","10,17%",8.333333,1
B3SA3 - 2023,01.11.2023,13.30,11.14,"13,45","11,13","729,75M","19,82%",19.389587,1
COGN3 - 2023,01.11.2023,3.23,2.41,"3,26","2,39","528,64M","34,58%",34.024896,1
...,...,...,...,...,...,...,...,...,...
BAHI3 - 2023,01.11.2023,8.06,9.15,"9,91","7,90","68,70K","-10,44%",-11.912568,0
NEXP3 - 2023,01.11.2023,5.36,4.93,"5,70","4,91","24,60K","8,72%",8.722110,1
EMAE4 - 2023,01.11.2023,59.51,59.39,"62,93","55,06","147,00K","0,02%",0.202054,1


# 2022

In [37]:
dataframe_variacao_preco_acoes_2022 = criacao_dataframe_dados_historicos('dados historicos', acoes_do_dataframe_2022, '01.11.2022', '2022')
dataframe_variacao_preco_acoes_2022

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Variação,Classificação
Empresa,,,,,,,,,
PETR4 - 2022,01.11.2022,22.39,22.46,"22,70","18,73","2,27B","2,22%",-0.311665,0
OIBR3 - 2022,01.11.2022,2.00,2.60,"2,90","1,50","287,05M","-20,00%",-23.076923,0
B3SA3 - 2022,01.11.2022,12.43,14.66,"15,60","11,01","1,45B","-15,36%",-15.211460,0
ITSA4 - 2022,01.11.2022,7.78,8.48,"8,70","7,45","733,91M","-8,58%",-8.254717,0
ABEV3 - 2022,01.11.2022,15.12,15.23,"16,04","14,25","577,69M","-0,50%",-0.722259,0
...,...,...,...,...,...,...,...,...,...
BOBR4 - 2022,01.11.2022,1.29,1.24,"1,40","1,19","764,10K","4,03%",4.032258,1
CRPG5 - 2022,01.11.2022,24.85,20.65,"30,60","17,91","583,90K","19,29%",20.338983,1
BAHI3 - 2022,01.11.2022,13.58,14.99,"16,09","12,69","254,33K","-12,32%",-9.406271,0


In [38]:
# Descobrindo o index que está presente em dataframe_3trimestres_2022 e não está presente em dataframe_variacao_preco_acoes_2022 por causa de dados faltantes
index = dataframe_3trimestres_2022.index.difference(dataframe_variacao_preco_acoes_2022.index)
# Retirando a linha de index do dataframe_3trimestres_2022
dataframe_3trimestres_2022 = dataframe_3trimestres_2022.drop(index)

# Concatenando os 2 dataframes

In [39]:
dataframe_3trimestres = pd.concat([dataframe_3trimestres_2022, dataframe_3trimestres_2023])
dataframe_3trimestres

,1T_Dívida Líquida,1T_EBIT,1T_Lucro Líquido,1T_ROE,1T_Receita Líquida,2T_Dívida Líquida,2T_EBIT,2T_Lucro Líquido,2T_ROE,2T_Receita Líquida,3T_Dívida Líquida,3T_EBIT,3T_Lucro Líquido,3T_ROE,3T_Receita Líquida
Empresa,,,,,,,,,,,,,,,
PETR4 - 2022,189.848.000.000,65.398.000.000,44.783.000.000,"10,26 %",141.641.000.000,180.367.000.000,96.434.000.000,54.484.000.000,"13,23 %",170.960.000.000,256.715.000.000,74.612.000.000,46.236.000.000,"12,37 %",170.076.000.000
OIBR3 - 2022,28.541.960.000,-369.370.000,1.783.950.000,"172,14 %",2.486.940.000,16.167.680.000,-611.400.000,-320.660.000,"-41,52 %",2.579.520.000,18.365.367.000,-992.420.000,-3.062.460.000,"-134,16 %",2.770.060.000
B3SA3 - 2022,-5.093.250.000,1.430.770.000,1.101.040.000,"5,05 %",2.284.700.000,-2.477.130.000,1.399.600.000,1.092.280.000,"5,29 %",2.241.630.000,-2.874.606.000,1.415.720.000,1.030.000.000,"5,04 %",2.510.000.000
ITSA4 - 2022,4.888.000.000,4.167.000.000,3.859.000.000,"5,67 %",2.131.000.000,5.137.000.000,3.270.000.000,3.181.000.000,"4,56 %",2.214.000.000,7.387.000.000,3.961.000.000,3.651.000.000,"5,02 %",2.161.000.000
ABEV3 - 2022,-11.247.310.000,4.162.330.000,3.528.840.000,"4,32 %",18.439.150.000,-12.468.933.000,4.034.080.000,3.064.050.000,"3,44 %",17.989.000.000,-16.079.870.000,4.054.970.000,3.215.010.000,"3,45 %",20.587.640.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAHI3 - 2023,177.193.000,-7.610.000,-17.872.000,"-30,31 %",86.779.000,167.306.000,1.138.000,-14.127.000,"-22,00 %",100.934.000,178.597.000,-597.000,-15.005.000,"-45,80 %",94.225.000
NEXP3 - 2023,48.337.000,-13.298.000,-17.843.000,"-18,42 %",30.066.000,50.187.000,26.464.000,29.871.000,"-23,20 %",37.420.000,50.369.000,4.650.000,4.411.000,"-4,09 %",-2.473.000
EMAE4 - 2023,-447.375.000,1.038.000,18.882.000,"1,65 %",137.552.000,-468.834.000,10.281.000,37.848.000,"3,39 %",138.754.000,-457.016.000,39.526.000,41.102.000,"3,55 %",173.596.000


In [40]:
dataframe_variacao_preco_acoes = pd.concat([dataframe_variacao_preco_acoes_2022, dataframe_variacao_preco_acoes_2023])
dataframe_variacao_preco_acoes

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Variação,Classificação
Empresa,,,,,,,,,
PETR4 - 2022,01.11.2022,22.39,22.46,"22,70","18,73","2,27B","2,22%",-0.311665,0
OIBR3 - 2022,01.11.2022,2.00,2.60,"2,90","1,50","287,05M","-20,00%",-23.076923,0
B3SA3 - 2022,01.11.2022,12.43,14.66,"15,60","11,01","1,45B","-15,36%",-15.211460,0
ITSA4 - 2022,01.11.2022,7.78,8.48,"8,70","7,45","733,91M","-8,58%",-8.254717,0
ABEV3 - 2022,01.11.2022,15.12,15.23,"16,04","14,25","577,69M","-0,50%",-0.722259,0
...,...,...,...,...,...,...,...,...,...
BAHI3 - 2023,01.11.2023,8.06,9.15,"9,91","7,90","68,70K","-10,44%",-11.912568,0
NEXP3 - 2023,01.11.2023,5.36,4.93,"5,70","4,91","24,60K","8,72%",8.722110,1
EMAE4 - 2023,01.11.2023,59.51,59.39,"62,93","55,06","147,00K","0,02%",0.202054,1


## Criando o dataframe com os resultados trimestrais e a classificação

In [41]:
dataframe_dados_completos = atribuir_classificacao(dataframe_3trimestres, dataframe_variacao_preco_acoes)
dataframe_dados_completos

,1T_Dívida Líquida,1T_EBIT,1T_Lucro Líquido,1T_ROE,1T_Receita Líquida,2T_Dívida Líquida,2T_EBIT,2T_Lucro Líquido,2T_ROE,2T_Receita Líquida,3T_Dívida Líquida,3T_EBIT,3T_Lucro Líquido,3T_ROE,3T_Receita Líquida,Classificação
Empresa,,,,,,,,,,,,,,,,
PETR4 - 2022,189.848.000.000,65.398.000.000,44.783.000.000,"10,26 %",141.641.000.000,180.367.000.000,96.434.000.000,54.484.000.000,"13,23 %",170.960.000.000,256.715.000.000,74.612.000.000,46.236.000.000,"12,37 %",170.076.000.000,0
OIBR3 - 2022,28.541.960.000,-369.370.000,1.783.950.000,"172,14 %",2.486.940.000,16.167.680.000,-611.400.000,-320.660.000,"-41,52 %",2.579.520.000,18.365.367.000,-992.420.000,-3.062.460.000,"-134,16 %",2.770.060.000,0
B3SA3 - 2022,-5.093.250.000,1.430.770.000,1.101.040.000,"5,05 %",2.284.700.000,-2.477.130.000,1.399.600.000,1.092.280.000,"5,29 %",2.241.630.000,-2.874.606.000,1.415.720.000,1.030.000.000,"5,04 %",2.510.000.000,0
ITSA4 - 2022,4.888.000.000,4.167.000.000,3.859.000.000,"5,67 %",2.131.000.000,5.137.000.000,3.270.000.000,3.181.000.000,"4,56 %",2.214.000.000,7.387.000.000,3.961.000.000,3.651.000.000,"5,02 %",2.161.000.000,0
ABEV3 - 2022,-11.247.310.000,4.162.330.000,3.528.840.000,"4,32 %",18.439.150.000,-12.468.933.000,4.034.080.000,3.064.050.000,"3,44 %",17.989.000.000,-16.079.870.000,4.054.970.000,3.215.010.000,"3,45 %",20.587.640.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAHI3 - 2023,177.193.000,-7.610.000,-17.872.000,"-30,31 %",86.779.000,167.306.000,1.138.000,-14.127.000,"-22,00 %",100.934.000,178.597.000,-597.000,-15.005.000,"-45,80 %",94.225.000,0
NEXP3 - 2023,48.337.000,-13.298.000,-17.843.000,"-18,42 %",30.066.000,50.187.000,26.464.000,29.871.000,"-23,20 %",37.420.000,50.369.000,4.650.000,4.411.000,"-4,09 %",-2.473.000,1
EMAE4 - 2023,-447.375.000,1.038.000,18.882.000,"1,65 %",137.552.000,-468.834.000,10.281.000,37.848.000,"3,39 %",138.754.000,-457.016.000,39.526.000,41.102.000,"3,55 %",173.596.000,1


In [42]:
# Colunas em que os valores serão mudados de 1.000.000 para 1000000
colunas_para_retirar_o_ponto = np.array(dataframe_dados_completos.columns)
colunas_para_retirar_o_ponto = np.delete(colunas_para_retirar_o_ponto, [3, 8, 13, 15])

### Realizando o tratamento dos dados do dataset

In [43]:
# Substituindo as '%' por '' e as ',' por '.' e convertendo para float
dataframe_dados_completos = dataframe_dados_completos.replace('%', '', regex=True)
dataframe_dados_completos = dataframe_dados_completos.replace(',', '.', regex=True)
# Agora eu preciso tirar os '.' para conseguir converter o numero para float, por exemplo, 1.000.000 para 1000000, isso deve ser feito em 'colunas_para_retirar_o_ponto'
for coluna in colunas_para_retirar_o_ponto:
    dataframe_dados_completos[coluna] = dataframe_dados_completos[coluna].str.replace('.', '')

dataframe_dados_completos = dataframe_dados_completos.astype(float)
#Convertento a coluna 'Classificação' para int
dataframe_dados_completos['Classificação'] = dataframe_dados_completos['Classificação'].astype(int)
dataframe_dados_completos


,1T_Dívida Líquida,1T_EBIT,1T_Lucro Líquido,1T_ROE,1T_Receita Líquida,2T_Dívida Líquida,2T_EBIT,2T_Lucro Líquido,2T_ROE,2T_Receita Líquida,3T_Dívida Líquida,3T_EBIT,3T_Lucro Líquido,3T_ROE,3T_Receita Líquida,Classificação
Empresa,,,,,,,,,,,,,,,,
PETR4 - 2022,1.898480e+11,6.539800e+10,4.478300e+10,10.26,1.416410e+11,1.803670e+11,9.643400e+10,5.448400e+10,13.23,1.709600e+11,2.567150e+11,7.461200e+10,4.623600e+10,12.37,1.700760e+11,0
OIBR3 - 2022,2.854196e+10,-3.693700e+08,1.783950e+09,172.14,2.486940e+09,1.616768e+10,-6.114000e+08,-3.206600e+08,-41.52,2.579520e+09,1.836537e+10,-9.924200e+08,-3.062460e+09,-134.16,2.770060e+09,0
B3SA3 - 2022,-5.093250e+09,1.430770e+09,1.101040e+09,5.05,2.284700e+09,-2.477130e+09,1.399600e+09,1.092280e+09,5.29,2.241630e+09,-2.874606e+09,1.415720e+09,1.030000e+09,5.04,2.510000e+09,0
ITSA4 - 2022,4.888000e+09,4.167000e+09,3.859000e+09,5.67,2.131000e+09,5.137000e+09,3.270000e+09,3.181000e+09,4.56,2.214000e+09,7.387000e+09,3.961000e+09,3.651000e+09,5.02,2.161000e+09,0
ABEV3 - 2022,-1.124731e+10,4.162330e+09,3.528840e+09,4.32,1.843915e+10,-1.246893e+10,4.034080e+09,3.064050e+09,3.44,1.798900e+10,-1.607987e+10,4.054970e+09,3.215010e+09,3.45,2.058764e+10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAHI3 - 2023,1.771930e+08,-7.610000e+06,-1.787200e+07,-30.31,8.677900e+07,1.673060e+08,1.138000e+06,-1.412700e+07,-22.00,1.009340e+08,1.785970e+08,-5.970000e+05,-1.500500e+07,-45.80,9.422500e+07,0
NEXP3 - 2023,4.833700e+07,-1.329800e+07,-1.784300e+07,-18.42,3.006600e+07,5.018700e+07,2.646400e+07,2.987100e+07,-23.20,3.742000e+07,5.036900e+07,4.650000e+06,4.411000e+06,-4.09,-2.473000e+06,1
EMAE4 - 2023,-4.473750e+08,1.038000e+06,1.888200e+07,1.65,1.375520e+08,-4.688340e+08,1.028100e+07,3.784800e+07,3.39,1.387540e+08,-4.570160e+08,3.952600e+07,4.110200e+07,3.55,1.735960e+08,1


In [44]:
# Salvando o dataframe em um arquivo CSV na pasta instancias modelos
dataframe_dados_completos.to_csv('instancias modelos/dataset_3T23_22.csv')

# Criando um dataset com dois trimestres para testes

In [45]:
# Excluindo as colunas que possuírem 3T do dataset dataframe_dados_completos e a coluna 'Classificação'
df_2t = dataframe_dados_completos.drop(dataframe_dados_completos.filter(like='3T').columns, axis=1)
df_2t = df_2t.drop('Classificação', axis=1)
df_2t

# Chamando a função criacao_dataframe_dados_historicos que vai ter a coluna classificação
dataframe_variacao_preco_acoes_2t_2023 = criacao_dataframe_dados_historicos('dados historicos', acoes_do_dataframe_2023, '01.08.2023', '2023')
dataframe_variacao_preco_acoes_2t_2022 = criacao_dataframe_dados_historicos('dados historicos', acoes_do_dataframe_2022, '01.08.2022', '2022')
dataframe_variacao_preco_acoes_2t = pd.concat([dataframe_variacao_preco_acoes_2t_2022, dataframe_variacao_preco_acoes_2t_2023])

# Chamando a função atribuir_classificacao
dataframe_dados_completos_2t = atribuir_classificacao(df_2t, dataframe_variacao_preco_acoes_2t)
dataframe_dados_completos_2t

,1T_Dívida Líquida,1T_EBIT,1T_Lucro Líquido,1T_ROE,1T_Receita Líquida,2T_Dívida Líquida,2T_EBIT,2T_Lucro Líquido,2T_ROE,2T_Receita Líquida,Classificação
Empresa,,,,,,,,,,,
PETR4 - 2022,1.898480e+11,6.539800e+10,4.478300e+10,10.26,1.416410e+11,1.803670e+11,9.643400e+10,5.448400e+10,13.23,1.709600e+11,1
OIBR3 - 2022,2.854196e+10,-3.693700e+08,1.783950e+09,172.14,2.486940e+09,1.616768e+10,-6.114000e+08,-3.206600e+08,-41.52,2.579520e+09,1
B3SA3 - 2022,-5.093250e+09,1.430770e+09,1.101040e+09,5.05,2.284700e+09,-2.477130e+09,1.399600e+09,1.092280e+09,5.29,2.241630e+09,1
ITSA4 - 2022,4.888000e+09,4.167000e+09,3.859000e+09,5.67,2.131000e+09,5.137000e+09,3.270000e+09,3.181000e+09,4.56,2.214000e+09,1
ABEV3 - 2022,-1.124731e+10,4.162330e+09,3.528840e+09,4.32,1.843915e+10,-1.246893e+10,4.034080e+09,3.064050e+09,3.44,1.798900e+10,1
...,...,...,...,...,...,...,...,...,...,...,...
BAHI3 - 2023,1.771930e+08,-7.610000e+06,-1.787200e+07,-30.31,8.677900e+07,1.673060e+08,1.138000e+06,-1.412700e+07,-22.00,1.009340e+08,0
NEXP3 - 2023,4.833700e+07,-1.329800e+07,-1.784300e+07,-18.42,3.006600e+07,5.018700e+07,2.646400e+07,2.987100e+07,-23.20,3.742000e+07,0
EMAE4 - 2023,-4.473750e+08,1.038000e+06,1.888200e+07,1.65,1.375520e+08,-4.688340e+08,1.028100e+07,3.784800e+07,3.39,1.387540e+08,0


In [46]:
dataframe_dados_completos_2t.to_csv('instancias modelos/dataset_2T23_22.csv')